In [1]:
import numpy as np
import pandas as pd 
import os 
import re 
from collections import Counter

In [2]:
data_path = '../dataset/Metal_Content_of_Consumer_Products_Tested_by_the_NYC_Health_Department_20240403.csv'
df = pd.read_csv(data_path)

In [4]:
def missspell(df):
    def read_glove_vecs(glove_file):
        with open(glove_file,'r') as f:
            words=set()
            for line in f:
                line=line.strip().split()
                curr_word=line[0]
                words.add(curr_word)
            i=1
            words_to_index={}
            for w in sorted(words):
                words_to_index[w]=i
                i=i+1
        del words
        return words_to_index
    WORDS=read_glove_vecs('dataset/glove.6B.50d.txt')
    def words(text):
        return re.findall(r'\w+', text.lower())
    def P(word):
        return -WORDS.get(word,0)
    def correction(word):
        return max(candidates(word), key=P)
    def candidates(word):
        c =  (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
        return c
    def known(ws):
        return set(w for w in ws if w in WORDS)
    def edits1(word):
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    def edits2(word):
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))
    def find_corrections(text):
        arr = text
        corrections_list = []
        for s in arr:
            s = s.lower()
            if s and correction(s) != s:
                corrections_list.append((s, correction(s)))
        return corrections_list
    corrections = []
    for index, line in df['MANUFACTURER'].items():
        row_corrections = find_corrections(line)
        corrections += row_corrections
        if row_corrections:
            df.at[index, 'MANUFACTURER'] = 'Misspell'
        else:
            df.at[index, 'MANUFACTURER'] = 'valid'
    return df

In [6]:
missspell(df)

,ROW_ID,PRODUCT_TYPE,PRODUCT_NAME,METAL,CONCENTRATION,UNITS,MANUFACTURER,MADE_IN_COUNTRY,COLLECTION_DATE,DELETED
0,1,Food-Spice,Turmeric powder,Lead,2.9,ppm,valid,INDIA,01/04/2011 12:00:00 AM,NaN
1,2,Food-Spice,Cumin powder,Lead,2.4,ppm,valid,INDIA,01/04/2011 12:00:00 AM,NaN
2,3,Food-Spice,Cumin seeds,Lead,2.8,ppm,valid,INDIA,01/04/2011 12:00:00 AM,NaN
3,4,Food-Spice,Masala,Lead,12.0,ppm,valid,UNKNOWN OR NOT STATED,01/04/2011 12:00:00 AM,NaN
4,5,Food-Candy,Tamarind Candy,Lead,-1.0,ppm,valid,UNKNOWN OR NOT STATED,01/06/2011 12:00:00 AM,NaN
...,...,...,...,...,...,...,...,...,...,...
6888,6889,Other,Mogra Incense Sticks,Lead,1.9,ppm,valid,INDIA,12/20/2022 12:00:00 AM,NaN
6889,6890,Food Other,Home made ground cereal,Lead,26.0,ppm,valid,UNKNOWN OR NOT STATED,12/20/2022 12:00:00 AM,NaN
6890,6891,Food-Candy,Chimoy,Lead,-1.0,ppm,valid,UNKNOWN OR NOT STATED,12/28/2022 12:00:00 AM,NaN
6891,6892,Food-Candy,Chocolate,Lead,-1.0,ppm,valid,UNITED STATES,12/28/2022 12:00:00 AM,NaN
